In [1]:
import os
import glob
import geopandas as gpd
import rasterio
from rasterio.mask import mask
import pandas as pd
import numpy as np
from tqdm import tqdm
from shapely.geometry import box, mapping

In [2]:
def load_claims(claims_dir):
    """
    Load and combine FEMA claims data from both auto and property claims.
    
    Parameters:
    -----------
    claims_dir : str
        Directory containing the FEMA claims shapefiles
        
    Returns:
    --------
    GeoDataFrame
        Combined claims data with geometry and claim information
    """
    print("Loading FEMA claims data...")
    
    # Load auto claims
    auto_claims_path = os.path.join(claims_dir, 'FEMA_Harvey2017_AutoClaims_Aug25_Sep08.shp')
    auto_claims = gpd.read_file(auto_claims_path).to_crs(epsg=4326)
    
    # Load property claims
    property_claims_path = os.path.join(claims_dir, 'FEMA_Harvey2017_PropertyClaims_Aug25_Sep08.shp')
    property_claims = gpd.read_file(property_claims_path).to_crs(epsg=4326)
    
    # Combine claims
    all_claims = pd.concat([auto_claims, property_claims], ignore_index=True)
    
    print(f"Loaded {len(auto_claims)} auto claims and {len(property_claims)} property claims")
    return all_claims

In [ ]:
claims_dir = '../FEMA_Harvey2017_Claims_shp'
flood_maps_pattern = '../data/HOU00*_500yr.tif'

# Find all flood maps
flood_maps = glob.glob(flood_maps_pattern)

# Load claims data
claims_gdf = load_claims(claims_dir)

Loading FEMA claims data...


In [ ]:
with rasterio.open('../data/HOU001_500yr.tif') as flood_src:
    # Get flood map bounds
    bounds = flood_src.bounds
    flood_bbox = gpd.GeoDataFrame(
        geometry=[box(bounds.left, bounds.bottom, bounds.right, bounds.top)],
        crs=flood_src.crs
    )
    
    # Convert claims to flood map CRS and crop to flood map bounds
    claims_crs = claims_gdf.to_crs(flood_src.crs)
    claims_cropped = gpd.overlay(claims_crs, flood_bbox, how='intersection')